In [1]:
import gumpy as gp
import numpy as np
import pywt

import sklearn
import os

import seaborn as sns

import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

from scipy import linalg

import pandas as pd

#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
from keras.utils import to_categorical

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

Using TensorFlow backend.


In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



# load data step

In [4]:
def load_data(filename):
    
    #extra_overlap = 1500
    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    #此通道没有采集 置为0
    #全通道均使用时 不需要
    #for i in range(len(SAMPLE_CHANNEL)):
    #    if SAMPLE_CHANNEL[i] == 'none':
    #        data[: , i] = 0.0

    #删除前x秒和后x秒数据
    
    
    #是否进行裁剪 【如果进行裁剪 由于sen的第一次数据 将extra_overlap调整为1500】
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    
    #data = np.concatenate((data , data[ -extra_overlap : , :]) , axis=0)
    #
    #data_filter = butter_worth(data , 0.5 , 40 , order=3)
    #
    #return data_filter[extra_overlap : , :] #将边界效应去掉

    return data

In [5]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

## method 1 sub band power

In [6]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

In [7]:
def alpha_subBP_features(data):
    alpha1 = butter_worth(data , 8.5 , 11.5)
    alpha2 = butter_worth(data , 9.0 , 12.5)    
    alpha3 = butter_worth(data , 9.5 , 13.5)   #11.5 后
    alpha4 = butter_worth(data , 8.0 , 10.5)   
    
    return np.array([alpha1 , alpha2 , alpha3 , alpha4])

def beta_subBP_features(data):
    beta1 = butter_worth(data , 15.0 , 30.0) #14.0 前
    beta2 = butter_worth(data , 16.0 , 17.0)    
    beta3 = butter_worth(data , 17.0 , 18.0)    
    beta4 = butter_worth(data , 18.0 , 19.0)    
    
    return np.array([beta1 , beta2 , beta3 , beta4])

def powermean(data):
    #官方demo跳4秒 前4秒为准备阶段
    return np.power(data[ : , 0] , 2).mean(), \
            np.power(data[ : , 1] , 2).mean(), \
            np.power(data[ : , 2] , 2).mean(), \
            np.power(data[ : , 3] , 2).mean(), \
            np.power(data[ : , 4] , 2).mean(), \
            np.power(data[ : , 5] , 2).mean(), \
            np.power(data[ : , 6] , 2).mean(), \
            np.power(data[ : , 7] , 2).mean(), \
            np.power(data[ : , 8] , 2).mean(), \
            np.power(data[ : , 9] , 2).mean(), \
            np.power(data[ : , 10] , 2).mean(), \
            np.power(data[ : , 11] , 2).mean(), \
            np.power(data[ : , 12] , 2).mean(), \
            np.power(data[ : , 13] , 2).mean(), \
            np.power(data[ : , 14] , 2).mean(), \
            np.power(data[ : , 15] , 2).mean()       

In [8]:
def log_subBP_feature_extraction(alpha , beta):
    #alpha
    power_1_a = powermean(alpha[0])
    power_2_a = powermean(alpha[1])
    power_3_a = powermean(alpha[2])
    power_4_a = powermean(alpha[3])
    
    #beta
    power_1_b = powermean(beta[0])
    power_2_b = powermean(beta[1])
    power_3_b = powermean(beta[2])
    power_4_b = powermean(beta[3])
    
    X= np.array(
        [np.log(power_1_a) ,
         np.log(power_2_a) ,
         np.log(power_3_a) ,
         np.log(power_4_a) ,
         np.log(power_1_b) ,
         np.log(power_2_b) ,
         np.log(power_3_b) ,
         np.log(power_4_b)
        ]
        ).flatten()

    return X

In [9]:
def feature_extraction_sub_band_power(data):
    n_features = 128
    X = np.zeros((data.shape[0] , n_features))
    
    for i , datum in enumerate(data):
        alpha = alpha_subBP_features(datum)
        beta = beta_subBP_features(datum)
            
        X[i, :] = log_subBP_feature_extraction(alpha , beta)

    return X

## method 2 DWT

In [10]:
def feature_extraction_dwt_meta(data , n):
    n_features = 48
    
    X = np.zeros((data.shape[0] , n_features))
    
    level = 5
    wavelet = 'db4'
    
    #n=3 or 4
    
    for i , datum in enumerate(data):
        coeffs_Pz  = pywt.wavedec(data = datum[:,0], wavelet=wavelet, level=level)
        coeffs_PO3 = pywt.wavedec(data = datum[:,1], wavelet=wavelet, level=level)
        coeffs_PO4 = pywt.wavedec(data = datum[:,2], wavelet=wavelet, level=level)
        coeffs_O1  = pywt.wavedec(data = datum[:,3], wavelet=wavelet, level=level)
        coeffs_O2  = pywt.wavedec(data = datum[:,4], wavelet=wavelet, level=level)
        coeffs_Oz  = pywt.wavedec(data = datum[:,5], wavelet=wavelet, level=level)
        coeffs_O9  = pywt.wavedec(data = datum[:,6], wavelet=wavelet, level=level)
        coeffs_FP2 = pywt.wavedec(data = datum[:,7], wavelet=wavelet, level=level)
        coeffs_C4  = pywt.wavedec(data = datum[:,8], wavelet=wavelet, level=level)
        coeffs_C6  = pywt.wavedec(data = datum[:,9], wavelet=wavelet, level=level)
        coeffs_CP3 = pywt.wavedec(data = datum[:,10], wavelet=wavelet, level=level)
        coeffs_CP1 = pywt.wavedec(data = datum[:,11], wavelet=wavelet, level=level)
        coeffs_CPZ = pywt.wavedec(data = datum[:,12], wavelet=wavelet, level=level)
        coeffs_CP2 = pywt.wavedec(data = datum[:,13], wavelet=wavelet, level=level)
        coeffs_CP4 = pywt.wavedec(data = datum[:,14], wavelet=wavelet, level=level)
        coeffs_PO8 = pywt.wavedec(data = datum[:,15], wavelet=wavelet, level=level)

        X[i , :] = np.array([
            np.std(coeffs_Pz [n]),   
            np.std(coeffs_PO3[n]),  
            np.std(coeffs_PO4[n]),   
            np.std(coeffs_O1 [n]),  
            np.std(coeffs_O2 [n]),  
            np.std(coeffs_Oz [n]),   
            np.std(coeffs_O9 [n]),  
            np.std(coeffs_FP2[n]),  
            np.std(coeffs_C4 [n]),
            np.std(coeffs_C6 [n]),
            np.std(coeffs_CP3[n]),
            np.std(coeffs_CP1[n]),
            np.std(coeffs_CPZ[n]),
            np.std(coeffs_CP2[n]),
            np.std(coeffs_CP4[n]),
            np.std(coeffs_PO8[n]),
            
            np.mean(coeffs_Pz [n]**2),
            np.mean(coeffs_PO3[n]**2),
            np.mean(coeffs_PO4[n]**2),
            np.mean(coeffs_O1 [n]**2),
            np.mean(coeffs_O2 [n]**2),
            np.mean(coeffs_Oz [n]**2),
            np.mean(coeffs_O9 [n]**2),
            np.mean(coeffs_FP2[n]**2),            
            np.mean(coeffs_C4 [n]**2),
            np.mean(coeffs_C6 [n]**2),
            np.mean(coeffs_CP3[n]**2),
            np.mean(coeffs_CP1[n]**2),
            np.mean(coeffs_CPZ[n]**2),
            np.mean(coeffs_CP2[n]**2),
            np.mean(coeffs_CP4[n]**2),
            np.mean(coeffs_PO8[n]**2),
            
            np.mean(coeffs_Pz [n]),
            np.mean(coeffs_PO3[n]), 
            np.mean(coeffs_PO4[n]),
            np.mean(coeffs_O1 [n]),
            np.mean(coeffs_O2 [n]),
            np.mean(coeffs_Oz [n]),
            np.mean(coeffs_O9 [n]),
            np.mean(coeffs_FP2[n]),        
            np.mean(coeffs_C4 [n]),
            np.mean(coeffs_C6 [n]),
            np.mean(coeffs_CP3[n]),
            np.mean(coeffs_CP1[n]),
            np.mean(coeffs_CPZ[n]),
            np.mean(coeffs_CP2[n]),
            np.mean(coeffs_CP4[n]),
            np.mean(coeffs_PO8[n])]).flatten()
        
    return X

def normalize(data , normalization_type = 'mean_std'):
    
    def _norm_mean_std(data):
        _mean = np.mean(data , axis=0)
        _std = np.std(data , axis=0)
        
        return (data - _mean) / _std
    
    def _norm_min_max(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    
    if normalization_type == 'mean_std':
        return _norm_mean_std(data)
    elif normalization_type == 'min_max':
        return _norm_min_max(data)
    else:
        raise Exception('wrong normalization type')
    
def feature_extraction_dwt(data , is_normalize = True):
    data_3 = feature_extraction_dwt_meta(data , 3) #4
    data_4 = feature_extraction_dwt_meta(data , 4) #5
    
    data_concat = np.concatenate((data_3 , data_4) , axis = -1)
    
    if is_normalize:
        return normalize(data_concat)
    else:
        return data_concat

# method 4 rms feature

In [11]:
def feature_extraction_RMS(data):
    def rms(datum):
        '''
        :datum: 一段信号 shape : (3*256) * 16
        '''
        return [ np.sqrt(np.mean(np.square( d ))) for d in datum.T ]
    
    feature_rms = []
    
    for datum in data:
        feature_rms.append(rms(datum))
    
    return np.array(feature_rms)

# classify step
# ensemble voting

In [12]:
def feature_selection(data , labels , model , num_features , cv=10):
    '''
    :model: classify model
    :num_features: features count you expect(integer or tuple)
    '''
    
    '''[8 20]'''
    
    sfs = SequentialFeatureSelector(model , k_features=num_features , cv=cv , verbose = 2 , n_jobs=-1) #all cpu cores
    
    sfs.fit(data , labels)
    
    #最优秀的特征索引
    return sfs.k_feature_idx_


def choose_common_feature_idx(data , labels , classifiers , use_ratio = 0.25 , num_features = 10 , num_features_threshold = 8):
    '''
    sub_band_power使用该函数 进行筛选特征
    :use_ratio: 0.25 ~= 360/1500
    :num_features:integer or tuple 期望的特征数量（待选择的数量）
    :min_num_features: 特征数量阈值 小于时 停止选择
    选择适合所有分类器的特征索引值
    集合 与 运算
    '''
    
    #将特征选择需要使用的数据 0.25 进行置乱
    idxes_ratio = np.random.randint(0 , data.shape[0] , size = int(use_ratio * data.shape[0]) )
    
    
    data_shuffle = data #[idxes_ratio]
    labels_shuffle = labels #[idxes_ratio]
    
    feature_idxes = set(list(range(data.shape[1]))) #初始化为所有的特征索引值
    
    #============
    random_idxes = np.random.randint(0 , data.shape[1] , size = 25)
    
    data_shuffle = data_shuffle[: , random_idxes]
    labels_shuffle = labels_shuffle #保持完整性
    #============
    
    for classifier in classifiers:
        
        idx = feature_selection(data_shuffle , labels_shuffle , classifier , num_features)
        idx = set(idx)
        
        #寻找共同的特征索引
        #寻找之前先测试 如果小于阈值 直接停止
        if len(feature_idxes & idx) < num_features_threshold:
            break
            
        feature_idxes = feature_idxes & idx
        
        break
        
    return np.array(list(feature_idxes))

# 匹配法
 
## 前向特征选择之后 使用匹配法进行

In [13]:
def train_val(data , ratio = 0.9):
    '''
    将数据分为 训练集 和 验证集
    '''
    
    seg = int(ratio * data.shape[0])
    
    return data[ : seg] , data[seg : ]

def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames


def combine_match(freq = 10):
    '''
    匹配法中使用
    训练数据与验证数据
    数据不进行置乱
    '''
    
    if freq not in [10 , 15 , 20 , 25]:
        print('freq must in 10,15,20,25')
        return 
    
    ratio = 0.9 #训练集的占比
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/8/%s/' % freq) )
    person_9_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/9/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('real_data/eeg_final/circle/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('real_data/eeg_final/circle/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('real_data/eeg_final/circle/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('real_data/eeg_final/circle/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('real_data/eeg_final/circle/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('real_data/eeg_final/circle/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('real_data/eeg_final/circle/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('real_data/eeg_final/circle/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('real_data/eeg_final/circle/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    person_9 = np.concatenate([load_data('real_data/eeg_final/circle/9/%s/' % freq + filename) for filename in person_9_filenames] , axis = 0)

    person_0_train , person_0_val = train_val(person_0)
    person_1_train , person_1_val = train_val(person_1)
    person_2_train , person_2_val = train_val(person_2)
    person_3_train , person_3_val = train_val(person_3)
    person_4_train , person_4_val = train_val(person_4)
    person_5_train , person_5_val = train_val(person_5)
    person_6_train , person_6_val = train_val(person_6)
    person_7_train , person_7_val = train_val(person_7)
    person_8_train , person_8_val = train_val(person_8)
    person_9_train , person_9_val = train_val(person_9)
    
    #数据分段阶段
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0_train , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1_train , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2_train , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3_train , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4_train , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5_train , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6_train , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7_train , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8_train , label = 8 , overlap_length=overlap_length)
    train_person_data_9 , train_person_labels_9 = separate(person_9_train , label = 9 , overlap_length=overlap_length)

    '''
    不进行置乱 不能进行置乱
    '''
    #合并数据
    train_data = [train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                  train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                  train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                  train_person_data_9]
    
    train_labels = [train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                    train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                    train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                    train_person_labels_9]
    
    #============
    #验证数据分段
    val_person_data_0 , val_person_labels_0 = separate(person_0_val , label = 0 , overlap_length=0)
    val_person_data_1 , val_person_labels_1 = separate(person_1_val , label = 1 , overlap_length=0)
    val_person_data_2 , val_person_labels_2 = separate(person_2_val , label = 2 , overlap_length=0)
    val_person_data_3 , val_person_labels_3 = separate(person_3_val , label = 3 , overlap_length=0)
    val_person_data_4 , val_person_labels_4 = separate(person_4_val , label = 4 , overlap_length=0)
    val_person_data_5 , val_person_labels_5 = separate(person_5_val , label = 5 , overlap_length=0)
    val_person_data_6 , val_person_labels_6 = separate(person_6_val , label = 6 , overlap_length=0)
    val_person_data_7 , val_person_labels_7 = separate(person_7_val , label = 7 , overlap_length=0)
    val_person_data_8 , val_person_labels_8 = separate(person_8_val , label = 8 , overlap_length=0)
    val_person_data_9 , val_person_labels_9 = separate(person_9_val , label = 9 , overlap_length=0)
    
    #合并数据
    val_data = [val_person_data_0 , val_person_data_1 , val_person_data_2 ,
                val_person_data_3 , val_person_data_4 , val_person_data_5 ,
                val_person_data_6 , val_person_data_7 , val_person_data_8 ,
                val_person_data_9]
    
    val_labels = [val_person_labels_0 , val_person_labels_1 , val_person_labels_2 ,
                  val_person_labels_3 , val_person_labels_4 , val_person_labels_5 ,
                  val_person_labels_6 , val_person_labels_7 , val_person_labels_8 ,
                  val_person_labels_9]

    return train_data , train_labels , val_data , val_labels


def con_mat_m(_real_labels , _labels):
    '''
    匹配方法使用 可视化
    :_real_labels: 真实label
    :_labels: 预测的label
    '''
    
    print('match')
    print('val score:%f' % ( np.sum( np.equal( np.concatenate(_real_labels) , np.concatenate(_labels) ) ) / len( np.concatenate(_labels) ) ) )
    print('real')
    
    print(confusion_matrix( np.concatenate(_real_labels) , np.concatenate(_labels) ))
    print(classification_report( np.concatenate(_real_labels) , np.concatenate(_labels) ))
    
def con_mat_heatmap(_labels_hat , _labels , color , png_path):
    
    _labels_hat = np.concatenate(_labels_hat)
    _labels = np.concatenate(_labels)
    
    mat = confusion_matrix( _labels , _labels_hat )
    
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False , cmap=color )#,
            #xticklabels=faces.target_names,
            #yticklabels=faces.target_names)
    
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    
    plt.xlabel('precision:%.2f recall:%.2f f1:%.2f' % (precision , recall , f1) )
    # plt.ylabel('predicted label');
    plt.savefig(png_path) #保存起来
    plt.close()

In [14]:
#相似度函数 越相似 相似度越小（为了容易使用欧氏距离与曼哈顿距离定义相似度） 直接使用距离来度量相似性

def match(database_data_sbp , oob_data_sbp , distance_type = 'c'):
    '''
    :database_data_sbp: 充当数据库
    :oob_data_sbp: 来进行匹配的
    '''
    
    distances = [] 
    
    if distance_type == 'o':
        #欧氏距离
        distance_method = euclidean_distances
    elif distance_type == 'm':
        #曼哈顿距离
        distance_method = manhattan_distances
    elif distance_type == 'c':
        #余弦距离
        distance_method = cosine_distances
    else:
        pass
    
    labels = [] #数据库中匹配的标记
    
    for oob_datum_sbp in oob_data_sbp: #
        
        labels_one_subject = []
        for oob_datum_sbp_sub in oob_datum_sbp:
            #进行匹配的 逐个计算
            distance = []
            for database_datum_sbp in database_data_sbp:
                #与每一个受试者的特征进行距离计算
                distance.append( np.mean( distance_method(oob_datum_sbp_sub , database_datum_sbp) ) ) #距离的平均值
                
            label = np.argmin(distance)
            
            labels_one_subject.append(label)
        
        labels.append(labels_one_subject)
    
    return labels

In [15]:
#不对RMS进行有特征选择的匹配实验
dir_name = ['sub-band power' , 'DWT']

for i , model in enumerate([feature_extraction_sub_band_power , feature_extraction_dwt]): #特征提取方式
    for freq in [10 , 15 , 20 , 25]: #频率
        for distance in ['o' , 'm' , 'c']: #距离度量方式
            for t in range(20): #测试20次
                
                #=================
                #初始化所有分类器
                svc = SVC(probability=True)
                rf =  RandomForestClassifier()
                adaboost = AdaBoostClassifier()
                xgb = xgboost.XGBClassifier()
                lgbm = lightgbm.LGBMClassifier()
                gbc = GradientBoostingClassifier()
                knn = KNeighborsClassifier()
                dt = DecisionTreeClassifier()
                lda = LDA()
                nb = GaussianNB()
                mlp = MLPClassifier()
                
                classifiers = [xgb , lgbm , gbc , rf , svc , dt , lda , adaboost , mlp , nb , knn ]
                #=================
                
                
                database_data , database_labels , oob_data , oob_labels = combine_match(freq = freq)
                
                database_data_sbp = [ model(database_datum) for database_datum in database_data ]
                oob_data_sbp = [ model(oob_datum) for oob_datum in oob_data ]
                
                #=========
                #特征选择
                chosen_idx = choose_common_feature_idx(np.concatenate(database_data_sbp) , np.concatenate(database_labels) , classifiers , use_ratio=0.25)                
                #=========
                
                #使用特征选择出来的特征索引 进行匹配
                database_data_sbp = [ database_datum_sbp[: , chosen_idx] for database_datum_sbp in database_data_sbp]
                oob_data_sbp = [ oob_datum_sbp[: , chosen_idx] for oob_datum_sbp in oob_data_sbp]
                
                oob_labels_hat = match(database_data_sbp , oob_data_sbp , distance_type=distance)
                
                con_mat_heatmap(oob_labels_hat , oob_labels , color=None ,
                                png_path = 'c://Users/qq122/Desktop/v1_paper_pic_matching/%s/有特征提取/%shz/%s/%s.png' % (dir_name[i] , freq , distance , t+1) )

[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   21.7s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   23.1s finished

[2018-12-24 15:43:55] Features: 1/10 -- score: 0.49666666666666676[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   22.4s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   24.6s finished

[2018-12-24 15:44:21] Features: 2/10 -- score: 0.6426666666666666[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   24.6s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   26.3s finished

[2018-12-24 15:44:48] Features: 3/10 -- score: 0.6846666666666666[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   27.5s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   29.6s finished

[2018-12-24 15:45:19] Features: 4/10 -- score: 0.6906666666666668[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   31.6s remaining:    7.4s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | e

[2018-12-24 16:30:02] Features: 6/10 -- score: 0.7033333333333334[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed:   31.4s remaining:   11.2s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:   36.1s finished

[2018-12-24 16:30:40] Features: 7/10 -- score: 0.7186666666666667[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:   30.1s remaining:   11.5s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   36.3s finished

[2018-12-24 16:31:18] Features: 8/10 -- score: 0.7193333333333334[Parallel(n_jobs=-1)]: Done  11 out of  17 | elapsed:   32.3s remaining:   17.6s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   39.2s finished

[2018-12-24 16:31:58] Features: 9/10 -- score: 0.7186666666666667[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   32.0s remaining:   19.2s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   39.4s finished

[2018-12-24 16:32:38] Features: 10/10 -- score: 0.7193333333333334[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   22.9s 

[2018-12-24 17:16:15] Features: 2/10 -- score: 0.5953333333333334[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   26.6s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   27.3s finished

[2018-12-24 17:16:43] Features: 3/10 -- score: 0.6133333333333333[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   27.9s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   28.7s finished

[2018-12-24 17:17:14] Features: 4/10 -- score: 0.6319999999999999[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   28.3s remaining:    6.6s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   29.9s finished

[2018-12-24 17:17:45] Features: 5/10 -- score: 0.6473333333333333[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   29.9s remaining:    7.4s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   31.5s finished

[2018-12-24 17:18:17] Features: 6/10 -- score: 0.6613333333333332[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed:   28.0s r

[2018-12-24 17:39:21] Features: 10/10 -- score: 0.7106666666666667[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   23.0s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   23.5s finished

[2018-12-24 17:40:20] Features: 1/10 -- score: 0.5359999999999999[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   24.6s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   26.0s finished

[2018-12-24 17:40:48] Features: 2/10 -- score: 0.666[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   26.7s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   28.3s finished

[2018-12-24 17:41:17] Features: 3/10 -- score: 0.7086666666666666[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   29.0s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   30.0s finished

[2018-12-24 17:41:49] Features: 4/10 -- score: 0.7246666666666666[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   31.2s remaining:   

[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   35.1s finished

[2018-12-24 18:03:07] Features: 9/10 -- score: 0.716[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   29.9s remaining:   17.9s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   36.5s finished

[2018-12-24 18:03:45] Features: 10/10 -- score: 0.716[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   23.2s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   23.6s finished

[2018-12-24 18:04:45] Features: 1/10 -- score: 0.5093333333333334[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   27.2s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   28.7s finished

[2018-12-24 18:05:15] Features: 2/10 -- score: 0.6413333333333334[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   30.8s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   31.9s finished

[2018-12-24 18:05:48] Features: 3/10 -- score: 0.6886666666666666[Parallel(n_jobs

...

[2018-12-26 11:22:01] Features: 1/10 -- score: 0.5346666666666667[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   22.1s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   24.5s finished

[2018-12-26 11:22:28] Features: 2/10 -- score: 0.716[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   24.4s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   26.0s finished

[2018-12-26 11:22:57] Features: 3/10 -- score: 0.7806666666666666[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   26.4s remaining:    4.1s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   27.5s finished

[2018-12-26 11:23:26] Features: 4/10 -- score: 0.8193333333333334[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   28.0s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   29.1s finished

[2018-12-26 11:23:58] Features: 5/10 -- score: 0.8466666666666667[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:   29.9s remaining:    

[2018-12-26 11:39:06] Features: 10/10 -- score: 0.992[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   18.2s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   19.6s finished

[2018-12-26 11:39:29] Features: 1/10 -- score: 0.9879999999999999[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   18.8s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   20.2s finished

[2018-12-26 11:39:51] Features: 2/10 -- score: 0.9886666666666667[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   19.6s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   20.4s finished

[2018-12-26 11:40:14] Features: 3/10 -- score: 0.9873333333333333[Parallel(n_jobs=-1)]: Done  19 out of  22 | elapsed:   20.8s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   20.9s finished

[2018-12-26 11:40:37] Features: 4/10 -- score: 0.9873333333333333[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:   21.0s remaining:   

[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   23.8s finished

[2018-12-26 11:54:51] Features: 9/10 -- score: 0.9893333333333333[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   20.1s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   24.4s finished

[2018-12-26 11:55:17] Features: 10/10 -- score: 0.9893333333333333[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:   18.1s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   19.6s finished

[2018-12-26 11:55:40] Features: 1/10 -- score: 0.9893333333333334[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:   18.8s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   20.3s finished

[2018-12-26 11:56:03] Features: 2/10 -- score: 0.9960000000000001[Parallel(n_jobs=-1)]: Done  20 out of  23 | elapsed:   19.8s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   20.3s finished

[2018-12-26 11:56:27] Features: 3/10 -- score: 0.996000

Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-d420fe0834cd>", line 28, in <module>
    database_data_sbp = [ model(database_datum) for database_datum in database_data ]
  File "<ipython-input-15-d420fe0834cd>", line 28, in <listcomp>
    database_data_sbp = [ model(database_datum) for database_datum in database_data ]
  File "<ipython-input-11-fcb4ac0aa939>", line 11, in feature_extraction_RMS
    feature_rms.append(rms(datum))
  File "<ipython-input-11-fcb4ac0aa939>", line 6, in rms
    return [ np.sqrt(np.mean(np.square( d ))) for d in datum.T ]
  File "<ipython-input-11-fcb4ac0aa939>", line 6, in <listcomp>
    return [ np.sqrt(np.mean(np.square( d ))) for d in datum.T ]
  File "C:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py", line 2957, in mean
    in the result as dimensions with size one. With this option,


KeyboardInterrupt: 